In [3]:
!pip install rouge datasets==1.4.1 transformers==4.4.2

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.


## preprocess data

In [1]:
import os

path = './demo_data/SUMMARY.hk01meta'
os.makedirs(path, exist_ok=True)
print("The new directory is created!")

The new directory is created!


In [5]:
## data process 
import pandas as pd
from datasets import load_dataset

df_article_summary_full = pd.read_parquet('/home/ec2-user/SageMaker/hk01/data/meta_description.parquet', engine='pyarrow')

df_article_summary_full[['original_text','meta_descrption']].to_csv('./demo_data/SUMMARY.hk01meta/total.csv',index=False)

total_data = pd.read_csv('./demo_data/SUMMARY.hk01meta/total.csv')
x = total_data[-total_data['meta_descrption'].isnull()]
x.columns = ['article','summarization']

In [9]:
# use csv file to test 
x[:1000].to_csv(os.path.join(path,'train.csv'),index=False,encoding='utf-8')
x[1000:1200].to_csv(os.path.join(path,'test.csv'),index=False,encoding='utf-8')
x[1200:1400].to_csv(os.path.join(path,'dev.csv'),index=False,encoding='utf-8')

## run train

In [23]:
%%time
!python run_gen_v2.py --model_path 'fnlp/cpt-large' --dataset hk01meta --data_dir demo_data --epoch '1' --batch_size '4' 

Using custom data configuration default-4fa5bad4c3e38839
Reusing dataset csv (/home/ec2-user/.cache/huggingface/datasets/csv/default-4fa5bad4c3e38839/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c)
dataset:  DatasetDict({
    train: Dataset({
        features: ['article', 'summarization'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['article', 'summarization'],
        num_rows: 200
    })
    test: Dataset({
        features: ['article', 'summarization'],
        num_rows: 200
    })
})
11/02/2021 07:39:14 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
11/02/2021 07:39:14 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='output/hk01meta/4', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluation_strategy=<IntervalStrategy.EPOCH: 'epoch'>, prediction_loss_only=False, per_device_train_batch_siz

## 推理

In [16]:
import sys
sys.path.append('../')

from transformers import BertTokenizer
from modeling_cpt import CPTModel, CPTForConditionalGeneration

model = CPTForConditionalGeneration.from_pretrained("output/hk01meta/2")
tokenizer = BertTokenizer.from_pretrained('output/hk01meta/2')
dataset = load_dataset('csv', data_files='./demo_data/SUMMARY.hk01meta/test.csv',split='train')

Using custom data configuration default-26371a9e5916b36a
Reusing dataset csv (/home/ec2-user/.cache/huggingface/datasets/csv/default-26371a9e5916b36a/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c)


In [19]:
%%time
def print_result(idx):
    input_text = dataset[idx]['article']
    print("input: ",input_text)

    inputs = tokenizer(input_text, return_tensors="pt",max_length=512)

    #prompt_length = len(tokenizer.decode(inputs['input_ids'][0]))
    #outputs = model.generate(inputs['input_ids'], max_length=64, do_sample=True, top_p=0.95, top_k=60)
    outputs = model.generate(inputs['input_ids'], max_length=64, top_p=0.95)
    generated = tokenizer.decode(outputs[0])

    print("prediction result: ",generated)
    
    print ('label: ',dataset[idx]['summarization'])

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs


In [21]:
print_result(3)

input:  【Pokemon 寶可夢 劍盾 攻略】除了主線、儲圖鑑、育成和對戰之外，原來劍盾中還有不少NPC支線事件 / 任務，完成後可拿到一些不錯的道具。 草路鎮尋寶事件 草路鎮內有一個黑肉小蘿莉跟你說有個尋寶迷題，解開後可獲得寶物，當然要把它解開了。謎面是「最初為草，由強至弱，追尋兩次」，簡單來說就是先找到草的石碑，然後再找它剋制的屬性之石碑兩次（即是水，然後是火），將三個石碑順序調查一次後就可獲得寶物（達人帶）了。 火的石碑在城鎮最右邊 拿到寶物後小蘿莉會說你太厲害了（意味深） 水舟鎮送外賣任務 （如果無法觸發可試試完成主線後再來）和水舟鎮的海鮮餐廳廚師對話，他們請你幫忙送外賣，總共要送3次，而每一次廚師都會說忘了問地址要玩家自己找目的地lol不過當然會有提示，而且地點 全都在水舟鎮 內。每次送完都會獲得道具，送完三次後可獲得「幸運蛋」，效果是持有的寶可夢獲得之經驗值提升。 第一次送外賣的提示是火車聲 十分簡單就是火車站旁的屋子 你唔係外賣仔！！ 第二次的提示是綿綿芙的叫聲 第二次的提示是綿綿芙的叫聲 答案是靠近樓梯的屋子，這家人養了一隻綿綿芙 答案是靠近樓梯的屋子，這家人養了一隻綿綿芙 答案是靠近樓梯的屋子，這家人養了一隻綿綿芙 答案是靠近樓梯的屋子，這家人養了一隻綿綿芙 獲得兩個巨大金珠作報酬 第三次的提示是綠色的屋頂 第三次的提示是綠色的屋頂 綠色屋頂是沒有但有帳篷頂，在市集那邊 綠色屋頂是沒有但有帳篷頂，在市集那邊 最後再回去和廚師對話可得幸運蛋 最後再回去和廚師對話可得幸運蛋 效果是攜帶的寶可夢獲得經驗值增加，但是對經驗糖果無效，用途有限。 機擎市尋找泡沫栗鼠 在機擎市上層酒店再往左少許有一對站在貨櫃前的雙胞胎，和左邊那位對話他會說他的泡沫栗鼠走失了，見義勇為的主角當然要幫他找回來啦！他說他的泡沫栗鼠喜歡水窪和口哨，原來是走了去道館左手邊的噴水池，在那裡吹一下口哨（按左Analog）便可以找回泡沫栗鼠。之後回去和小男孩對話可獲得「爽喉噴霧」作為謝禮。 拳關市送信任務 第一次來到拳關市時，可以在火車站旁邊的樓梯上去，能在一個小女孩NPC手上接過一封信，請你交給舞姿鎮的法蘭克。在舞姿鎮找到法蘭克時卻發現他是個大叔，收到信時說很懷念，原來信是他多年前還在孤兒院時的玩伴「寶拉」寫的；但後來法蘭克離開了孤兒院（現在連孫兒都有了，生活應該很好），之後就沒再和

## 增强训练

In [7]:
%%time
!python run_gen_v2.py --model_path 'output/hk01meta/2' --dataset hk01meta --data_dir demo_data --epoch '1' --batch_size '4' 

Using custom data configuration default-4fa5bad4c3e38839
Reusing dataset csv (/home/ec2-user/.cache/huggingface/datasets/csv/default-4fa5bad4c3e38839/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c)
dataset:  DatasetDict({
    train: Dataset({
        features: ['article', 'summarization'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['article', 'summarization'],
        num_rows: 200
    })
    test: Dataset({
        features: ['article', 'summarization'],
        num_rows: 200
    })
})
11/02/2021 07:46:26 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
11/02/2021 07:46:26 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='output/hk01meta/4', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluation_strategy=<IntervalStrategy.EPOCH: 'epoch'>, prediction_loss_only=False, per_device_train_batch_siz